In [1]:
import argparse
import os
import re
import shutil
import subprocess
import sys

In [2]:
# Append JASPAR-profile-inference to path
jaspar_dir = os.path.abspath("./JASPAR-profile-inference")
sys.path.insert(0, jaspar_dir)
sys.path

['/home/ofornes/Work/GRECO/JASPAR-profile-inference',
 '/home/ofornes/Work/GRECO',
 '/home/ofornes/Work/Anaconda3/envs/JASPAR-profile-inference/lib/python37.zip',
 '/home/ofornes/Work/Anaconda3/envs/JASPAR-profile-inference/lib/python3.7',
 '/home/ofornes/Work/Anaconda3/envs/JASPAR-profile-inference/lib/python3.7/lib-dynload',
 '',
 '/home/ofornes/Work/Anaconda3/envs/JASPAR-profile-inference/lib/python3.7/site-packages',
 '/home/ofornes/Work/Anaconda3/envs/JASPAR-profile-inference/lib/python3.7/site-packages/IPython/extensions',
 '/home/ofornes/.ipython']

In [3]:
# Import from JASPAR-profile-inference
from __init__ import Jglobals

In [4]:
#-------------#
# Class       #
#-------------#

class TF(object):

    def __init__(self, gene_name, species):

        self.gene_name = gene_name
        self.species = species
        self.uniacc = None
        self.unientry = None
        self.status = None
        self.sequence = None
        self.family = "Unknown"
        self.cluster_num = None
        # self.orthodb = set()
        self.jaspar_id = None
        self.hocomoco_id = set()
        
        # In vivo
        self.chip_atlas = set()
        self.cistromedb = set()
        self.gtrd = set()
        self.dap_seq = set()
        self.remap = set()

        # In vitro
        self.ht_selex = set()
        self.cisbp = set()
        self.uniprobe = set()
        self.smile_seq = set()

        # Hidden variables (for internal use only)
        self._uniaccs = set()
        self._unientries = set()
        self._sequences = set()

    @property
    def invivo(self):
        """
        Returns 1 if the TF has been profiled by in vivo methods,
        or 0 otherwise.
        @rtype = {int}
        """

        # For simplicity, ChIP-seq ~ ChIP-exo ~ DAP-seq
        if self.chip_atlas or self.cistromedb or self.gtrd or self.dap_seq or self.remap:
            return(1)

        return(0)

    @property
    def invitro(self):
        """
        Returns the number of different experimental methods by
        which a TF has been profiled in vitro.
        @rtype = {int}
        """

        n = 0

        if self.ht_selex:
            n += 1
        # i.e. PBM
        if self.cisbp or self.uniprobe:
            n += 1
        if self.smile_seq:
            n += 1

        return(n)

    @property
    def evidence(self):
        """
        Returns the amount of evidence associated with a TF.
        @rtype = {int}
        """
        return(self.invivo+self.invitro)

    def __str__(self):

        string = "{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(
            self.gene_name,
            self.species,
            self.uniacc,
            # ";".join(sorted([i for i in self._uniaccs if i != self.uniacc])),
            self.unientry,
            # ";".join(sorted([i for i in self._unientries if i != self.unientry])),
            self.status,
            self.sequence,
            # ";".join(sorted([i for i in self._sequences if i != self.sequence])),
            self.family,
            self.cluster_num,
            self.evidence,
            # ";".join(sorted([i for i in self._pfam_ids if i != self.pfam_id])),
            # ";".join(sorted(self.orthodb)),
            self.jaspar_id,
            ";".join(sorted(self.hocomoco_id)),
            ";".join(sorted(self.chip_atlas)),
            ";".join(sorted(self.cistromedb)),
            ";".join(sorted(self.gtrd)),
            ";".join(sorted(self.remap)),
            ";".join(sorted(self.dap_seq)),
            ";".join(sorted(self.ht_selex)),
            ";".join(sorted(self.cisbp)),
            ";".join(sorted(self.uniprobe)),
            ";".join(sorted(self.smile_seq))
        )

        return(string)

    def __repr__(self):

        string = "{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(
            self.gene_name,
            self.species,
            self.uniacc,
            # ";".join(sorted([i for i in self._uniaccs if i != self.uniacc])),
            self.unientry,
            # ";".join(sorted([i for i in self._unientries if i != self.unientry])),
            self.status,
            self.sequence,
            # ";".join(sorted([i for i in self._sequences if i != self.sequence])),
            self.family,
            self.cluster_num,
            self.evidence,
            # ";".join(sorted([i for i in self._pfam_ids if i != self.pfam_id])),
            # ";".join(sorted(self.orthodb)),
            self.jaspar_id,
            ";".join(sorted(self.hocomoco_id)),
            ";".join(sorted(self.chip_atlas)),
            ";".join(sorted(self.cistromedb)),
            ";".join(sorted(self.gtrd)),
            ";".join(sorted(self.remap)),
            ";".join(sorted(self.dap_seq)),
            ";".join(sorted(self.ht_selex)),
            ";".join(sorted(self.cisbp)),
            ";".join(sorted(self.uniprobe)),
            ";".join(sorted(self.smile_seq))
        )

        return(string)

In [5]:
#-------------#
# Paths       #
#-------------#

hocomoco_file = "./Data/Databases/HOCOMOCO/HOCOMOCOv11_full_jaspar_format.txt"
chip_atlas_file = "./Data/Experiments/ChIP-seq.ChIP-Atlas.tsv"
cistromedb_file = "./Data/Experiments/ChIP-seq.CistromeDB.tsv"
gtrd_file = "./Data/Experiments/ChIP-seq.GTRD.tsv"
remap_file = "./Data/Experiments/ChIP-seq.ReMap2020.tsv"
dap_seq_file = "./Data/Experiments/DAP-seq.PMID:27203113.tsv"
ht_selex_files = ["./Data/Experiments/HT-SELEX.PMID:23332764.tsv",
                  "./Data/Experiments/HT-SELEX.PMID:28473536.tsv"]
cisbp_txt_file = "./Data/Databases/CisBP-2.0/PWMs.txt"
cisbp_tsv_file = "./Data/Experiments/PBM.CisBP-2.0.tsv"
uniprobe_file = "./Data/Experiments/PBM.UniPROBE.tsv"
smile_seq = "./Data/Experiments/SMiLE-seq.PMID:28092692.tsv"

In [6]:
#-------------#
# Parse Data  #
#-------------#

# Initialize
tfs = set()

# For each line...
for line in Jglobals.parse_tsv_file("./Data/Parsed/TFs.tab.gz"):

    # Initialize
    tf = TF(line[0], line[1])

    # Get UniProt Accession
    uniaccs = line[2].split(";")
    tf.uniacc = uniaccs[0]
    tf._uniaccs.update(set(uniaccs))

    # Get UniProt Entry
    unientries = line[3].split(";")
    tf.unientry = unientries[0]
    tf._unientries.update(set(unientries))

    # Get sequence
    sequences = line[4].split(";")
    tf.sequence = sequences[0]
    tf._sequences.update(set(sequences))

    # Get status (i.e. reviewed or not)
    tf.status = line[5]

    # Get family
    tf.family = line[6]

    # Get JASPAR ids
    tf.jaspar_id = line[7]

    # # Get orthoDB cluster
    # codec = coreapi.codecs.CoreJSONCodec()
    # for uniacc in tf._uniaccs:
    #     json_file = os.path.join(args.orthodb, "%s.json" % uniacc)
    #     if not os.path.exists(json_file):
    #         client = coreapi.Client()
    #         response = client.get(
    #             "https://www.orthodb.org/search?query=%s&level=2759&species=2759" % uniacc)
    #         json_obj = json.loads(codec.encode(response))
    #         with open(json_file, "w") as j:
    #             j.write(json.dumps(json_obj, sort_keys=True, indent=4, separators=(",", ": ")))
    #     with open(json_file, "r") as j:  
    #         json_obj = json.load(j)
    #         for orthodb in json_obj["data"]:
    #             tf.orthodb.add(orthodb)

    # Add TF to TFs
    tfs.add(tf)

print(next(iter(tfs)))

At1g23810	Arabidopsis thaliana	Q9ZUB5	Q9ZUB5_ARATH	Unreviewed	MAGGSKSPASSLEDGKAYVNAVKVALEEAEPAKYQEFLRLFHEVIARRMGMATFSARMQDLLKDHPSLCLGLNVMLAPEYQRAIPPEASEEFHKVVGRSVPRPEPTIDDATSYLIAVKEAFHDEPAKYEEMLKLLNDFKARRVNAASVIARVEELMKDHSNLLFGFCVFLSATTSFTTKLKAKFQGDGSQVVDSVLQIMRMYGEGNKSKHDAYQEIVALVQGHDDLVMELSQIFTDPSTRV	Unknown	None	0	nan										


In [7]:
#-------------#
# HOCOMOCO    #
#-------------#

#  678 Homo sapiens
#  451 Mus musculus

# SMCA5_MOUSE # Not a TF: SWI/SNF-related matrix-associated actin-dependent regulator of chromatin subfamily A member 5
# FUBP1_MOUSE # Not a TF: Far upstream element-binding protein 1
# BRCA1_MOUSE # Not a TF: Breast cancer type 1 susceptibility protein homolog
# EVI1_MOUSE  # Not a TF: Histone-lysine N-methyltransferase MECOM
# TAF1_MOUSE  # Not a TF: Transcription initiation factor TFIID subunit 1
# BRAC_MOUSE  # Not a valid UniProt Entry
# HLTF_MOUSE  # Not a TF: Helicase-like transcription factor
# TAF1_HUMAN  # Not a TF: Transcription initiation factor TFIID subunit 1
# HLTF_HUMAN  # Not a TF: Helicase-like transcription factor
# BRCA1_HUMAN # Not a TF: Breast cancer type 1 susceptibility protein
# SMCA5_HUMAN # Not a TF: SWI/SNF-related matrix-associated actin-dependent regulator of chromatin subfamily A member 5
# ZF64A_HUMAN # Not a valid UniProt Entry
# BRAC_HUMAN  # Not a valid UniProt Entry
# EVI1_HUMAN  # Not a TF: Histone-lysine N-methyltransferase MECOM
# FUBP1_HUMAN # Not a TF: Far upstream element-binding protein 1
# BPTF_HUMAN  # Not a TF: Nucleosome-remodeling factor subunit BPTF
# CENPB_HUMAN # Not a TF: Major centromere autoantigen B
# ZBT48_HUMAN # Not a valid UniProt Entry; should be TZAP_HUMAN

# For each line...
for line in Jglobals.parse_file(hocomoco_file):

    if line.startswith(">"):

        # Get unientry
        m = re.search("(\w+_(HUMAN|MOUSE))", line)
        unientry = m.group(1)

        # For each TF...
        for tf in sorted(tfs, key=lambda x: x.gene_name):
            if unientry in tf._unientries:
                tf.hocomoco_id.add(line[1:])

In [8]:
genes = 0
feats = 0
count = 0
for tf in sorted(tfs, key=lambda x: x.gene_name):
    subtotal_feats = len(tf.hocomoco_id)
    if subtotal_feats > 0:
        genes += 1
        feats += subtotal_feats
        if count < 10:
            print(tf.gene_name, tf.hocomoco_id)
        elif count == 10:
            print("...")
        else:
            pass
        count += 1
print("//")
print("Total genes: %s" % genes)
print("Total feats: %s" % feats)

AHR {'AHR_HUMAN.H11MO.0.B'}
AIRE {'AIRE_HUMAN.H11MO.0.C'}
ALX1 {'ALX1_HUMAN.H11MO.0.B'}
ALX3 {'ALX3_HUMAN.H11MO.0.D'}
ALX4 {'ALX4_HUMAN.H11MO.0.D'}
AR {'ANDR_HUMAN.H11MO.0.A', 'ANDR_HUMAN.H11MO.1.A', 'ANDR_HUMAN.H11MO.2.A'}
ARID3A {'ARI3A_HUMAN.H11MO.0.D'}
ARID5B {'ARI5B_HUMAN.H11MO.0.C'}
ARNT {'ARNT_HUMAN.H11MO.0.B'}
ARNT2 {'ARNT2_HUMAN.H11MO.0.D'}
...
//
Total genes: 1057
Total feats: 1214


In [9]:
#-------------#
# GTRD        #
#-------------#

#   71 Arabidopsis thaliana
#  213 Caenorhabditis elegans
#   11 Danio rerio
#  249 Drosophila melanogaster
# 1236 Homo sapiens
#  513 Mus musculus
#   12 Rattus norvegicus
#  137 Saccharomyces cerevisiae
#   32 Schizosaccharomyces pombe

# For each line...
for line in Jglobals.parse_tsv_file(gtrd_file):

    # Inialize
    experiment_id = line[0]
    uniacc = line[1]

    # For each TF...
    for tf in sorted(tfs, key=lambda x: x.gene_name):
        if uniacc in tf._uniaccs:
            tf.gtrd.add(experiment_id)

In [10]:
genes = 0
feats = 0
count = 0
for tf in sorted(tfs, key=lambda x: x.gene_name):
    subtotal_feats = len(tf.gtrd)
    if subtotal_feats > 0:
        genes += 1
        feats += subtotal_feats
        if count < 10:
            print(tf.gene_name, tf.gtrd)
        elif count == 10:
            print("...")
        else:
            pass
        count += 1
print("//")
print("Total genes: %s" % genes)
print("Total feats: %s" % feats)

11723 {'EXP043557'}
ABF4 {'EXP041418'}
ADD1 {'EXP044073'}
ADNP {'EXP039553'}
AEBP2 {'EXP040109'}
AGL8 {'EXP040838'}
AP1 {'EXP041282'}
AR {'EXP038636', 'EXP038189', 'EXP040661', 'EXP038106', 'EXP040665', 'EXP038192', 'EXP038188', 'EXP038193', 'EXP037263', 'EXP040660', 'EXP038191', 'EXP036887', 'EXP037067'}
ARF6 {'EXP041307'}
ARID2 {'EXP040161'}
...
//
Total genes: 814
Total feats: 1958


In [11]:
#-------------#
# DAP-seq     #
#-------------#

# For each line...
for line in Jglobals.parse_tsv_file(dap_seq_file):

    if line[0] == "AvgSpotLen":
        continue

    species = line[8]
    sra_run = line[9]
    gene = line[15]

    if line[8] not in species:
        continue

    # For each TF...
    for tf in sorted(tfs, key=lambda x: x.gene_name):
        if gene.upper() in tf.gene_name.upper():
            tf.dap_seq.add(sra_run)

In [12]:
genes = 0
feats = 0
count = 0
for tf in sorted(tfs, key=lambda x: x.gene_name):
    subtotal_feats = len(tf.dap_seq)
    if subtotal_feats > 0:
        genes += 1
        feats += subtotal_feats
        if count < 10:
            print(tf.gene_name, tf.dap_seq)
        elif count == 10:
            print("...")
        else:
            pass
        count += 1
print("//")
print("Total genes: %s" % genes)
print("Total feats: %s" % feats)

ABF2 {'SRR2926839'}
ABF2 {'SRR2926839'}
ABI5 {'SRR2926840', 'SRR2926841'}
ABR1 {'SRR2926082', 'SRR2926081'}
AGL13 {'SRR2926463'}
AGL15 {'SRR2926465', 'SRR2926464'}
AGL16 {'SRR2926466', 'SRR2926467'}
AGL42 {'SRR2926470'}
AGL6 {'SRR2926472'}
AGL61 {'SRR2926472'}
...
//
Total genes: 474
Total feats: 911


In [13]:
#-------------#
# HT-SELEX    #
#-------------#

# For each file...
for file_name in ht_selex_files:

    m = re.search("HT-SELEX.PMID:(\d+).tsv", file_name)
    pmid = int(m.group(1))

    # For each line...
    for line in Jglobals.parse_tsv_file(file_name):

        if line[0] == "Alias":
            continue

        m = re.search("^([A-Za-z\d]+)_", line[0])
        if m:

            gene_name = m.group(1)

            if pmid == 23332764:
                sra_run = line[14]
            else:
                sra_run = line[19]

            # For each TF...
            for tf in sorted(tfs, key=lambda x: x.gene_name):
                # HT-SELEX data only available for human and mouse
                if tf.species not in ["Homo sapiens", "Mus musculus"]:
                    continue
                if tf.gene_name == gene_name:
                    tf.ht_selex.add(sra_run)

In [14]:
genes = 0
feats = 0
count = 0
for tf in sorted(tfs, key=lambda x: x.gene_name):
    subtotal_feats = len(tf.ht_selex)
    if subtotal_feats > 0:
        genes += 1
        feats += subtotal_feats
        if count < 10:
            print(tf.gene_name, tf.ht_selex)
        elif count == 10:
            print("...")
        else:
            pass
        count += 1
print("//")
print("Total genes: %s" % genes)
print("Total feats: %s" % feats)

ALX1 {'ERR1002054', 'ERR1002050', 'ERR1002056', 'ERR1002053', 'ERR1002051', 'ERR1002055', 'ERR1002052', 'ERR1002057'}
ALX3 {'ERR1002071', 'ERR1002061', 'ERR1002064', 'ERR195525', 'ERR1002073', 'ERR1002066', 'ERR193771', 'ERR1002058', 'ERR1002067', 'ERR193772', 'ERR195526', 'ERR1010299', 'ERR1002059', 'ERR1002072', 'ERR1010665', 'ERR193773', 'ERR1002060', 'ERR1002069', 'ERR1002070', 'ERR1011404', 'ERR1002062', 'ERR195527', 'ERR1011038', 'ERR1002065', 'ERR195528', 'ERR1002063', 'ERR193770', 'ERR1002068'}
ALX4 {'ERR1002075', 'ERR1002077', 'ERR194917', 'ERR1002080', 'ERR1002081', 'ERR1002076', 'ERR1002078', 'ERR1002079', 'ERR194920', 'ERR1002074', 'ERR194919', 'ERR194918'}
AR {'ERR193972', 'ERR193678', 'ERR193679', 'ERR193970', 'ERR193680', 'ERR193971', 'ERR193973', 'ERR193681'}
ARGFX {'ERR1002086', 'ERR1002082', 'ERR1002085', 'ERR1002087', 'ERR1002088', 'ERR1002084', 'ERR1002083', 'ERR1002089'}
ARNT2 {'ERR1002094', 'ERR1010666', 'ERR1002093', 'ERR1010300', 'ERR1002092', 'ERR1002090', 'ERR

In [15]:
#-------------#
# CIS-BP      #
#-------------#

valid_matrix_ids = set()

# Get valid PWMs
for matrix_id in Jglobals.parse_file(cisbp_txt_file):
    valid_matrix_ids.add(matrix_id)

# For each line...
for line in Jglobals.parse_file(cisbp_tsv_file):

    matrix_ids = re.findall("(M\d{5}_2.00)", line)

    if matrix_ids:

        line = line.strip("\n").split("\t")
        gene_name = line[1]
        species = line[2].replace("_", " ")

        # Skip inferred TFs
        if line[3] != "D":
            continue

        # For each TF...
        for tf in sorted(tfs, key=lambda x: x.gene_name):
            if tf.gene_name.upper() == gene_name.upper() and species == tf.species:
                tf.cisbp.update(valid_matrix_ids.intersection(set(matrix_ids)))

In [16]:
genes = 0
feats = 0
count = 0
for tf in sorted(tfs, key=lambda x: x.gene_name):
    subtotal_feats = len(tf.cisbp)
    if subtotal_feats > 0:
        genes += 1
        feats += subtotal_feats
        if count < 10:
            print(tf.gene_name, tf.cisbp)
        elif count == 10:
            print("...")
        else:
            pass
        count += 1
print("//")
print("Total genes: %s" % genes)
print("Total feats: %s" % feats)

ABF1 {'M01772_2.00', 'M01773_2.00'}
ABF1 {'M00001_2.00', 'M00902_2.00'}
ABF2 {'M01771_2.00'}
ABF2 {'M00069_2.00'}
ABF3 {'M01782_2.00'}
ABF4 {'M01057_2.00', 'M01056_2.00'}
ABI5 {'M01776_2.00'}
ABR1 {'M01051_2.00'}
ACE2 {'M00033_2.00'}
ADR1 {'M00021_2.00'}
...
//
Total genes: 1199
Total feats: 1541


In [17]:
#-------------#
# UniPROBE    #
#-------------#

# For each line...
for line in Jglobals.parse_tsv_file(uniprobe_file):
   
    if line[0] == "Protein":
        continue

    gene_name = line[0]
    uniprobe_id = line[1]

    # For each TF...
    for tf in sorted(tfs, key=lambda x: x.gene_name):
        if tf.gene_name.upper() == gene_name.upper() and line[2] == tf.species:
            tf.uniprobe.add(uniprobe_id)

In [18]:
genes = 0
feats = 0
count = 0
for tf in sorted(tfs, key=lambda x: x.gene_name):
    subtotal_feats = len(tf.uniprobe)
    if subtotal_feats > 0:
        genes += 1
        feats += subtotal_feats
        if count < 10:
            print(tf.gene_name, tf.uniprobe)
        elif count == 10:
            print("...")
        else:
            pass
        count += 1
print("//")
print("Total genes: %s" % genes)
print("Total feats: %s" % feats)

ABF1 {'UP00452'}
AFT1 {'UP00344'}
ARO80 {'UP00329'}
ARX {'UP00584'}
ASG1 {'UP00350'}
Abd-B {'UP00503'}
Ahctf1 {'UP01352'}
Alx3 {'UP00108'}
Alx4 {'UP00187'}
Ar {'UP01353'}
...
//
Total genes: 540
Total feats: 558


In [19]:
#-------------#
# SMiLE-seq   #
#-------------#

synonyms = {
    "CEBPb": "CEBPB",
    "cFOS": "FOS",
    "cFOSL2": "FOSL2",
    "cJUN": "JUN",
    "PPARa": "PPARA",
    "PPARg": "PPARG",
    "RXRa": "RXRA",
    "RXRg": "RXRG"            
}

# For each line...
for line in Jglobals.parse_tsv_file(smile_seq):

    if line[0] == "Assay_Type":
        continue

    sra_run = line[13]

    for gene_name in line[16].split("_")[0].split("-"):
        if gene_name in synonyms:
            gene_name = synonyms[gene_name]

        # For each TF...
        for tf in sorted(tfs, key=lambda x: x.gene_name):
            if tf.gene_name.upper() == gene_name.upper() and line[12] == tf.species:
                tf.smile_seq.add(sra_run)

In [20]:
genes = 0
feats = 0
count = 0
for tf in sorted(tfs, key=lambda x: x.gene_name):
    subtotal_feats = len(tf.smile_seq)
    if subtotal_feats > 0:
        genes += 1
        feats += subtotal_feats
        if count < 10:
            print(tf.gene_name, tf.smile_seq)
        elif count == 10:
            print("...")
        else:
            pass
        count += 1
print("//")
print("Total genes: %s" % genes)
print("Total feats: %s" % feats)

ARNTL {'SRR3405117', 'SRR3405116'}
CEBPB {'SRR3405054'}
CLOCK {'SRR3405117', 'SRR3405116'}
CTCF {'SRR3405055', 'SRR3405078', 'SRR3405066'}
EN1 {'SRR3405089'}
Egr1 {'SRR3402436'}
FLI1 {'SRR3405101'}
FOS {'SRR3405133', 'SRR3405142', 'SRR3405143', 'SRR3405150', 'SRR3405132'}
FOSB {'SRR3405144', 'SRR3405145', 'SRR3405151', 'SRR3405134'}
FOSL1 {'SRR3405136', 'SRR3405152', 'SRR3405146'}
...
//
Total genes: 70
Total feats: 151


In [21]:
#-------------#
# Triples     #
#-------------#
count = 0
evidence = {}
for tf in sorted(tfs, key=lambda x: x.evidence, reverse=True):
    if tf.invivo > 0 and tf.invitro > 1:
        evidence.setdefault(tf.species, 0)
        evidence[tf.species] += 1
        if count < 10:
            print(tf.gene_name, tf.species, tf.evidence, tf.invivo, tf.invitro)
        elif count == 10:
            print("...")
        else:
            pass
        count += 1
print("//")
for species in sorted(evidence):
    print(species, evidence[species])

JUN Homo sapiens 4 1 3
PAX7 Homo sapiens 4 1 3
MAX Homo sapiens 4 1 3
Rxra Mus musculus 4 1 3
FOS Homo sapiens 4 1 3
BCL11B Homo sapiens 3 1 2
BCL6 Homo sapiens 3 1 2
CEBPB Homo sapiens 3 1 2
CLOCK Homo sapiens 3 1 2
CTCF Homo sapiens 3 1 2
...
//
Homo sapiens 34
Mus musculus 9


In [22]:
#-------------#
# Duos        #
#-------------#
count = 0
evidence = {}
for tf in sorted(tfs, key=lambda x: x.evidence, reverse=True):
    if tf.invivo > 0 and tf.invitro > 0:
        evidence.setdefault(tf.species, 0)
        evidence[tf.species] += 1
        if count < 10:
            print(tf.gene_name, tf.species, tf.evidence, tf.invivo, tf.invitro)
        elif count == 10:
            print("...")
        else:
            pass
        count += 1
print("//")
for species in sorted(evidence):
    print(species, evidence[species])

JUN Homo sapiens 4 1 3
PAX7 Homo sapiens 4 1 3
MAX Homo sapiens 4 1 3
Rxra Mus musculus 4 1 3
FOS Homo sapiens 4 1 3
BCL11B Homo sapiens 3 1 2
BCL6 Homo sapiens 3 1 2
CEBPB Homo sapiens 3 1 2
CLOCK Homo sapiens 3 1 2
CTCF Homo sapiens 3 1 2
...
//
Arabidopsis thaliana 114
Caenorhabditis elegans 26
Drosophila melanogaster 28
Homo sapiens 214
Mus musculus 58
Saccharomyces cerevisiae 15


In [23]:
#-------------#
# Pfam        #
#-------------#

# The following code is adapted from:
# https://github.com/wassermanlab/JASPAR-profile-inference/blob/master/files/get_files.py

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
import json
from infer_profile import hmmAlign, hmmScan, _makeSeqFile

# Initialize
seq_file = ".seq.fasta"
hmm_database = os.path.join(jaspar_dir, "files", "pfam-DBDs", "all_DBDs.hmm")

# Change dir
os.chdir(sys.path[1])
os.chdir(os.path.abspath("./Data/Pfam/"))

# Skip if JSON file already exists
json_file = "TFs.json"
if not os.path.exists(json_file):

    # Initialize
    pfams = {}

    # For each TF...
    for tf in sorted(tfs, key=lambda x: x.gene_name):

        # Initialize
        uniacc = tf.uniacc
        pfams.setdefault(uniacc, [])

        # Make seq file
        seq = Seq(tf.sequence, IUPAC.protein)
        seq_record = SeqRecord(seq, id=uniacc, name=uniacc, description=uniacc)
        _makeSeqFile(seq_record, seq_file)

        # For each DBD...
        for pfam_ac, start, end, evalue in hmmScan(seq_file, hmm_database, non_overlapping_domains=True):

            # Initialize
            hmm_file = os.path.join(jaspar_dir, "files", "pfam-DBDs", "%s.hmm" % pfam_ac)

            # Make seq file
            sub_seq = seq[start:end]
            seq_record = SeqRecord(sub_seq, id=uniacc, name=uniacc, description=uniacc)
            _makeSeqFile(seq_record, seq_file)

            # Add DBDs
            alignment = hmmAlign(seq_file, hmm_file)
            pfams[uniacc].append((pfam_ac, alignment, start+1, end, evalue))

    # Write
    Jglobals.write(json_file, json.dumps(pfams, sort_keys=True, indent=4, separators=(",", ": ")))

    # Remove seq file
    if os.path.exists(seq_file):
        os.remove(seq_file)

# Change dir back
os.chdir(sys.path[1])

In [26]:
#-------------#
# Cluster     #
#-------------#

# The following code is adapted from:
# https://github.com/wassermanlab/JASPAR-profile-inference/blob/master/finfer_profile.py

import math
import numpy as np
from infer_profile import _fetchXs, _filter_results_by_Rost, _removeLowercase

def get_members(tf):

    # Initialize
    members = []
    tf_DBDs = [dbd[0] for dbd in pfams[tf.uniacc]]
    tf_alignments = [dbd[1] for dbd in pfams[tf.uniacc]]
    seq = Seq(tf.sequence, IUPAC.protein)
    seq_record = SeqRecord(seq, id=tf.uniacc, name=tf.uniacc, description=tf.uniacc)
    fasta_file = os.path.join(clusters_dir, "%s.fasta" % species_dict[tf.species])

    # Get cut-offs on the percentage of sequence identity
    cutoffs = {}
    for pfam_ac in pfam_cutoffs:
        if pfam_cutoffs[pfam_ac][0] in tf_DBDs:
            cutoffs.setdefault(pfam_cutoffs[pfam_ac][0], pfam_cutoffs[pfam_ac][1])

    # BLAST+ search
    blast_results = BLAST(seq_record, fasta_file)

    # Filter results
    for filtered_result in sorted(_filter_results_by_Rost(blast_results), key=lambda x: x[5], reverse=True):

        if filtered_result[1] in clusters:
            continue

        # Both TFs have same DBD composition
        if [dbd[0] for dbd in pfams[filtered_result[1]]] == tf_DBDs:

            # Inference: percentage of sequence identity
            pids = []
            pid_cutoffs = []
            alignments = [dbd[1] for dbd in pfams[filtered_result[1]]]
            for a in range(len(alignments)):
                s1 = _removeLowercase(tf_alignments[a])
                s2 = _removeLowercase(alignments[a])
                pids.append(sum(_fetchXs(s1, s2))/float(len(s1)))
                pid_cutoffs.append(pids[-1] >= cutoffs[tf_DBDs[a]])
            if True in pid_cutoffs:
                members.append(filtered_result[1])

    return(members)

def BLAST(seq_record, fasta_file):

    # Initialize
    blast_results = set()
    outfmt = "sseqid pident length qstart qend sstart send evalue bitscore ppos qlen slen"

    # Run BLAST+
    cmd = "blastp -db %s -outfmt \"6 %s\"" % (fasta_file, outfmt)
    process = subprocess.Popen([cmd], shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    fasta_sequence = ">%s\n%s" % (seq_record.id, seq_record.seq)
    process.stdin.write(fasta_sequence.encode())
    (blast_records, blast_errors) = process.communicate()

    # For each BLAST+ record...
    for blast_record in blast_records.decode("utf-8").split("\n"):

        # Custom BLAST+ record:
        # (1) identifier of target sequence;
        # (2) percentage of identical matches;
        # (3) alignment length;
        # (4-5, 6-7) start and end-position in query and in target;
        # (8) E-value;
        # (9) bit score;
        # (10) percentage of positive-scoring matches; and
        # (4-7, 11, 12) joint coverage (i.e. square root of the coverage
        # on the query and the target).
        blast_record = blast_record.split("\t")

        # Skip if not a BLAST+ record
        if len(blast_record) != 12: continue

        # Get BLAST+ record
        target_id = blast_record[0]
        percent_identities = float(blast_record[1])
        alignment_length = int(blast_record[2])
        query_start_end = "%s-%s" % (blast_record[3], blast_record[4])
        target_start_end = "%s-%s" % (blast_record[5], blast_record[6])
        e_value = float(blast_record[7])
        score = float(blast_record[8])
        percent_similarity = float(blast_record[9])
        query_aligned_residues = int(blast_record[4]) - int(blast_record[3]) + 1
        query_length = float(blast_record[10])
        target_aligned_residues = int(blast_record[6]) - int(blast_record[5]) + 1
        target_length = float(blast_record[11])
        query_coverage = query_aligned_residues * 100 / query_length
        target_coverage = target_aligned_residues * 100 / target_length
        joint_coverage = math.sqrt(query_coverage * target_coverage)

        # Add BLAST+ record to search results
        blast_results.add((seq_record.id, target_id, query_start_end, target_start_end, e_value, score, percent_identities, alignment_length, percent_similarity, joint_coverage))

    # Return results sorted by score
    return(list(sorted(blast_results, key=lambda x: x[-1], reverse=True)))

# Initialize
cluster = 0
clusters = {}
clusters_dir = "./Data/Clusters/"
species_dict = {
    "Arabidopsis thaliana": "plants",
    "Caenorhabditis elegans": "nematodes",
    "Drosophila melanogaster": "insects",
    "Homo sapiens": "vertebrates",
    "Mus musculus": "vertebrates",
    "Saccharomyces cerevisiae": "fungi"
}
with open("./Data/Pfam/TFs.json") as f:
    pfams = json.load(f)
with open(os.path.join(jaspar_dir, "files", "pfam-DBDs.json")) as f:
    pfam_cutoffs = json.load(f)

for species in species_dict:

    # Initialize
    fasta_file = os.path.join(clusters_dir, "%s.fasta" % species_dict[species])

    if not os.path.exists("%s.psq" % fasta_file):

        # For each TF...
        for tf in sorted(tfs, key=lambda x: x.gene_name):

            if tf.species != species:
                continue

            Jglobals.write(fasta_file, ">%s\n%s" % (tf.uniacc, tf.sequence))

for species in species_dict:

    # Initialize
    fasta_file = os.path.join(clusters_dir, "%s.fasta" % species_dict[species])

    if not os.path.exists("%s.psq" % fasta_file):

        # Make BLAST+ database
        cmd = "makeblastdb -in %s -dbtype prot" % fasta_file
        process = subprocess.run([cmd], shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Skip if JSON file already exists
json_file = os.path.join(clusters_dir, "clusters.json")
if not os.path.exists(json_file):

    # Start with TFs supported by both in vivo and in vitro evidence
    for tf in sorted(tfs, key=lambda x: x.evidence, reverse=True):

        if tf.uniacc in clusters:
            continue

        if tf.invivo > 0 and tf.invitro > 0:

            # Get cluster members
            members = get_members(tf)

            if members:

                cluster += 1

                for member in members:
                    clusters.setdefault(member, cluster)

    # Then focus on TFs supported by one kind of evidence
    for tf in sorted(tfs, key=lambda x: x.evidence, reverse=True):

        if tf.uniacc in clusters:
            continue

        if tf.evidence > 0:

            # Get cluster members
            members = get_members(tf)

            if members:

                cluster += 1

                for member in members:
                    clusters.setdefault(member, cluster)

    # Write
    Jglobals.write(json_file, json.dumps(clusters, sort_keys=True, indent=4, separators=(",", ": ")))

with open(json_file) as f:
    clusters = json.load(f)

for tf in sorted(tfs, key=lambda x: x.gene_name):
    if tf.uniacc in clusters:
        tf.cluster_num = clusters[tf.uniacc]

In [27]:
for tf in sorted(tfs, key=lambda x: x.gene_name):
    Jglobals.write("./GRECO.tsv", tf)

JUN	Homo sapiens	P05412	JUN_HUMAN	Reviewed	MTAKMETTFYDDALNASFLPSESGPYGYSNPKILKQSMTLNLADPVGSLKPHLRAKNSDLLTSPDVGLLKLASPELERLIIQSSNGHITTTPTPTQFLCPKNVTDEQEGFAEGFVRALAELHSQNTLPSVTSAAQPVNGAGMVAPAVASVAGGSGSGGFSASLHSEPPVYANLSNFNPGALSSGGGAPSYGAAGLAFPAQPQQQQQPPHHLPQQMPVQHPRLQALKEEPQTVPEMPGETPPLSPIDMESQERIKAERKRMRNRIAASKCRKRKLERIARLEEKVKTLKAQNSELASTANMLREQVAQLKQKVMNHVNSGCQLMLTQQLQTF	bZIP	1	4	MA0099.2;MA0099.3;MA0462.1;MA0462.2;MA0488.1;MA0489.1;MA1126.1;MA1127.1;MA1128.1;MA1129.1;MA1130.1;MA1131.1;MA1132.1;MA1133.1	JUN_HUMAN.H11MO.0.A			EXP037742;EXP038429;EXP039416;EXP039444;EXP039488;EXP039496;EXP039507;EXP039519;EXP039545;EXP039637;EXP039869;EXP039941;EXP039950;EXP039981;EXP040017;EXP040243;EXP040248;EXP040263;EXP040265;EXP040320;EXP040322			ERR1004354;ERR1004355;ERR1004356;ERR1004357;ERR1004358;ERR1004359;ERR1004360;ERR1004361		UP00426	SRR3405057;SRR3405058;SRR3405059;SRR3405132;SRR3405133;SRR3405134;SRR3405136;SRR3405137;SRR3405138;SRR3405140
PAX7	Homo sapiens	P23759	PAX7_HUMAN	Reviewed	MAAL